# Generative AI POC – Exploración de Datos IFC

Este notebook forma parte de una Prueba de Concepto (POC) cuyo objetivo es
evaluar el uso de Inteligencia Artificial Generativa (GenAI) para comprender
datos técnicos de proyectos inmobiliarios.

A partir de un archivo IFC de ejemplo, se extrae información estructurada
del proyecto (espacios, muros, puertas, ventanas) que luego puede ser
consultada mediante lenguaje natural usando modelos de GenAI.

Este notebook **no representa una solución productiva**, sino un artefacto
demostrativo para un workshop de Applied Generative AI.


## 1. Carga del archivo IFC

En esta sección se carga un archivo IFC de ejemplo y se inicializa el modelo
utilizando `ifcopenshell`.


In [6]:
import ifcopenshell

model = ifcopenshell.open("../data/sample.ifc")
print("Modelo cargado correctamente")


Modelo cargado correctamente


## 2. Exploración de datos del proyecto

A continuación se exploran los elementos principales del modelo IFC
para identificar espacios y componentes relevantes del proyecto.


In [7]:
# Obtener todos los espacios del modelo IFC
spaces = model.by_type("IfcSpace")

print(f"Número total de espacios encontrados: {len(spaces)}")

for i, space in enumerate(spaces[:10], start=1):
    name = space.LongName or space.Name
    print(f"{i}. {name}")


Número total de espacios encontrados: 21
1. Foyer
2. Hallway
3. Bathroom 1
4. Foyer
5. Hallway
6. Utility
7. Utility
8. Stair
9. Room
10. Roof


In [8]:
# Crear un resumen textual del proyecto (input para GenAI)
space_names = []

for space in spaces:
    name = space.LongName or space.Name
    if name:
        space_names.append(name)

summary_text = (
    f"Este proyecto inmobiliario contiene {len(space_names)} espacios. "
    f"Los principales ambientes identificados son: "
    + ", ".join(space_names[:10])
)

print(summary_text)


Este proyecto inmobiliario contiene 21 espacios. Los principales ambientes identificados son: Foyer, Hallway, Bathroom 1, Foyer, Hallway, Utility, Utility, Stair, Room, Roof


In [9]:
walls = model.by_type("IfcWall")
doors = model.by_type("IfcDoor")
windows = model.by_type("IfcWindow")

print(f"Muros: {len(walls)}")
print(f"Puertas: {len(doors)}")
print(f"Ventanas: {len(windows)}")


Muros: 57
Puertas: 14
Ventanas: 24


## 3. Resumen del proyecto para consultas con GenAI

En esta sección se genera un resumen textual del proyecto inmobiliario
a partir de los datos extraídos del archivo IFC.

Este resumen será utilizado como **entrada (contexto)** para modelos
de Inteligencia Artificial Generativa, permitiendo responder preguntas
en lenguaje natural sobre el proyecto.


In [10]:
project_summary = f"""
Resumen del proyecto inmobiliario:

- Número total de espacios: {len(spaces)}
- Número de muros: {len(walls)}
- Número de puertas: {len(doors)}
- Número de ventanas: {len(windows)}

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.
"""

print(project_summary)



Resumen del proyecto inmobiliario:

- Número total de espacios: 21
- Número de muros: 57
- Número de puertas: 14
- Número de ventanas: 24

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.



## 4. Interacción con GenAI mediante prompt

A continuación se presenta el prompt utilizado para interactuar con un modelo
de Inteligencia Artificial Generativa. Este prompt incluye el contexto del
proyecto inmobiliario y define las instrucciones necesarias para responder
preguntas en lenguaje natural sin inventar información.


Eres un asistente experto en proyectos inmobiliarios y construcción.

A continuación se te proporciona información técnica extraída automáticamente
desde un archivo IFC de un proyecto inmobiliario.

Tu tarea es:
- Interpretar esta información técnica
- Responder preguntas en lenguaje natural de forma clara y comprensible
- No inventar datos que no estén presentes en la información proporcionada

Información del proyecto:
---
Resumen del proyecto inmobiliario:

- Número total de espacios: 21
- Número de muros: 57
- Número de puertas: 14
- Número de ventanas: 24

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.
---

Instrucciones adicionales:
- Si una pregunta no puede responderse con la información disponible, indícalo claramente.
- Usa un lenguaje claro, orientado a usuarios no técnicos.
- Prioriza explicaciones concisas pero completas.

Responde la siguiente pregunta:

¿Puedes darme un resumen general del proyecto inmobiliario?


## Limitaciones del POC

Esta Prueba de Concepto tiene un alcance limitado y controlado. En particular:

- Se utilizan archivos IFC de ejemplo y datos sintéticos.
- No se integran datos reales de clientes ni información sensible.
- El análisis se basa en información disponible en el archivo IFC, sin enriquecimiento externo.
- No se incluye una interfaz de usuario ni despliegue productivo.
- El modelo de GenAI no se entrena ni ajusta específicamente para este dominio.

Estas limitaciones son intencionales y coherentes con el objetivo del POC:
validar la viabilidad técnica y el valor del enfoque propuesto.


## Siguientes pasos

Como evolución de esta Prueba de Concepto, se plantean los siguientes pasos:

- Integrar el flujo con servicios de Generative AI en Google Cloud (por ejemplo, Vertex AI).
- Automatizar la generación del resumen del proyecto como entrada al modelo.
- Refinar los prompts para distintos perfiles de usuario (ventas, técnico, ejecutivo).
- Evaluar la incorporación de otros datasets o fuentes de información.
- Explorar una interfaz simple para demostraciones más avanzadas.

Estos pasos permitirían avanzar progresivamente hacia un prototipo más completo,
sin comprometer aún una implementación productiva.


## Conclusión

Este notebook demuestra que es posible transformar datos técnicos de un
archivo IFC en información comprensible que puede ser utilizada por
modelos de Inteligencia Artificial Generativa para responder preguntas
sobre proyectos inmobiliarios.
